# Convert the TensorFlow-Trained BERT Model to a PyTorch Model

# Install TensorFlow and Transformers

In [2]:
!pip install -q tensorflow==2.1.0 --upgrade --ignore-installed
!pip install -q torch==1.5.0 --upgrade --ignore-installed
!pip install -q transformers==2.8.0

ERROR: awscli 1.18.39 has requirement botocore==1.15.39, but you'll have botocore 1.17.14 which is incompatible.
ERROR: awscli 1.18.39 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.6 which is incompatible.


In [3]:
!pip install torchserve torch-model-archiver

     |████████████████████████████████| 4.5 MB 18.1 MB/s eta 0:00:01


# Restart the Kernel

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [34]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Retrieve the Training Job Name

In [6]:
%store -r training_job_name

In [7]:
print('Previous training_job_name: {}'.format(training_job_name))

Previous training_job_name: tensorflow-training-2020-06-30-21-24-43-479


# Download the TensorFlow-Trained Model Locally

In [8]:
models_dir = './models'

In [9]:
# Download the model and output artifacts from AWS S3
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

fatal error: Parameter validation failed:
Invalid bucket name "": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:s3:[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$"


In [10]:
import tarfile
import pickle as pkl

#!ls -al ./models

tar = tarfile.open('{}/model.tar.gz'.format(models_dir))
tar.extractall(path=models_dir)
tar.close()

In [11]:
!ls -al $models_dir

total 487060
drwxrwxr-x 6 ec2-user ec2-user      4096 Jul  4 18:56 .
drwxrwxr-x 9 ec2-user ec2-user      4096 Jul  4 18:58 ..
drwxr-xr-x 2 ec2-user ec2-user      4096 Jun 30 21:34 metrics
-rw-rw-r-- 1 ec2-user ec2-user 498719004 Jun 30 21:35 model.tar.gz
drwxr-xr-x 2 ec2-user ec2-user      4096 Jun 30 21:29 tensorboard
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun 30 21:29 tensorflow
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun 30 21:29 transformers


In [12]:
transformer_model_dir = '{}/transformers/fine-tuned/'.format(models_dir)

!ls -al $transformer_model_dir

total 261692
drwxr-xr-x 2 ec2-user ec2-user      4096 Jun 30 21:33 .
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun 30 21:29 ..
-rw-r--r-- 1 ec2-user ec2-user      1358 Jun 30 21:33 config.json
-rw-r--r-- 1 ec2-user ec2-user 267959068 Jun 30 21:33 tf_model.h5


In [13]:
cat $transformer_model_dir/config.json

{
  "_num_labels": 5,
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true

# Convert the TensorFlow Model to PyTorch

In [14]:
from transformers import DistilBertForSequenceClassification # PyTorch version

loaded_pytorch_model = DistilBertForSequenceClassification.from_pretrained(transformer_model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     },
                                                                  from_tf=True)

In [15]:
print(type(loaded_pytorch_model))
print(loaded_pytorch_model)

<class 'transformers.modeling_distilbert.DistilBertForSequenceClassification'>
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [16]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inference_device = -1 # CPU: -1, GPU: 0
print('inference_device {}'.format(inference_device))

inference_device -1


In [17]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=loaded_pytorch_model, 
                                                tokenizer=tokenizer,
                                                framework='pt',
                                                device=inference_device) 

In [18]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

I loved it!  I will recommend this to everyone. [{'label': 5, 'score': 0.2354256}]


In [19]:
review = """Really bad.  I hope they don't make this anymore."""
print(review, inference_pipeline(review))

Really bad.  I hope they don't make this anymore. [{'label': 5, 'score': 0.23714961}]


# Save The Transformer/PyTorch Model with `.save_pretrained()`

In [20]:
pytorch_models_dir = './models/transformers/pytorch'

In [24]:
!mkdir -p $pytorch_models_dir

In [25]:
loaded_pytorch_model.save_pretrained(pytorch_models_dir)

In [26]:
!ls -al $pytorch_models_dir

total 261588
drwxrwxr-x 2 ec2-user ec2-user      4096 Jul  4 19:00 .
drwxr-xr-x 4 ec2-user ec2-user      4096 Jul  4 19:00 ..
-rw-rw-r-- 1 ec2-user ec2-user      1302 Jul  4 19:00 config.json
-rw-rw-r-- 1 ec2-user ec2-user 267852933 Jul  4 19:00 pytorch_model.bin


# Load and Predict

In [27]:
pytorch_model = DistilBertForSequenceClassification.from_pretrained(pytorch_models_dir)

In [28]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=pytorch_model, 
                                                tokenizer=tokenizer,
                                                framework='pt',
                                                device=inference_device) 

In [29]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

I loved it!  I will recommend this to everyone. [{'label': 5, 'score': 0.2354256}]


# Upload Transformer/PyTorch Model to S3

In [30]:
transformer_pytorch_model_name = 'pytorch_model.bin'

In [35]:
s3_transformer_pytorch_model_path = 's3://{}/models/transformer-pytorch/'.format(bucket)
print(s3_transformer_pytorch_model_path)

s3://sagemaker-us-east-1-835319576252/models/transformer-pytorch/


In [36]:
!aws s3 cp --recursive $pytorch_models_dir $s3_transformer_pytorch_model_path

upload: models/transformers/pytorch/config.json to s3://sagemaker-us-east-1-835319576252/models/transformer-pytorch/config.json
upload: models/transformers/pytorch/pytorch_model.bin to s3://sagemaker-us-east-1-835319576252/models/transformer-pytorch/pytorch_model.bin


In [37]:
!aws s3 ls $s3_transformer_pytorch_model_path

2020-07-04 19:01:20       1302 config.json
2020-07-04 19:01:20  267852933 pytorch_model.bin


In [38]:
%store s3_transformer_pytorch_model_path

Stored 's3_transformer_pytorch_model_path' (str)


# Save PyTorch Model with `torch.save()`

In [39]:
pytorch_model_name = 'pytorch_model.pt'

In [42]:
!mkdir -p ./models/pytorch/

In [43]:
pytorch_model_path = './models/pytorch/{}'.format(pytorch_model_name)

In [45]:
import torch
torch.save(loaded_pytorch_model.state_dict(), pytorch_model_path)

In [46]:
!ls -al $pytorch_model_path

-rw-rw-r-- 1 ec2-user ec2-user 267852933 Jul  4 19:02 ./models/pytorch/pytorch_model.pt


# Upload PyTorch Model to S3

In [47]:
s3_pytorch_model_path = 's3://{}/models/pytorch/{}'.format(bucket, pytorch_model_name)
print(s3_pytorch_model_path)

s3://sagemaker-us-east-1-835319576252/models/pytorch/pytorch_model.pt


In [48]:
!aws s3 cp $pytorch_model_path $s3_pytorch_model_path

upload: models/pytorch/pytorch_model.pt to s3://sagemaker-us-east-1-835319576252/models/pytorch/pytorch_model.pt


In [49]:
!aws s3 ls $s3_pytorch_model_path

2020-07-04 19:02:54  267852933 pytorch_model.pt


In [50]:
%store s3_pytorch_model_path

Stored 's3_pytorch_model_path' (str)
